In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [3]:
spark = SparkSession.builder.appName("spark_sql").getOrCreate()

25/07/19 06:30:46 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
actor_films_df = spark.read.format("csv")\
                        .option("header","true")\
                        .option("inferschema","true")\
                        .load("file:/home/iceberg/data/actor_films.csv")

In [5]:
actor_films_df.createOrReplaceTempView("actor_films")

In [ ]:
# actor dataset grouped for year 1980 and customized quality_class
spark.sql("""
select actor, year,
		array_agg( map(
          'film', film,
          'votes', cast(votes as string),
          'rating', cast(rating as string),
          'filmid', cast(filmid as string))) as films,
		case when avg(rating) > 8 then 'star'
		when avg(rating) > 7 and avg(rating) <= 8 then 'good'
		when avg(rating) > 6 and avg(rating) <= 7 then 'average'
		when avg(rating) <= 6 then 'bad'
	end as quality_class
from actor_films 
group by actor, year
""").show(5, truncate=False)

[Stage 13:>                                                         (0 + 1) / 1]

+----------------+----+----------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|actor           |year|films                                                                                                                                                     |quality_class|
+----------------+----+----------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|Adam Baldwin    |1980|[{film -> Ordinary People, votes -> 47657, rating -> 7.7, filmid -> tt0081283}, {film -> My Bodyguard, votes -> 8730, rating -> 7.1, filmid -> tt0081207}]|good         |
|Adam West       |1980|[{film -> The Happy Hooker Goes Hollywood, votes -> 413, rating -> 3.5, filmid -> tt0080839}]                                                             |bad          |
|Adrienne Barbeau|1980|[{film -> Th

In [ ]:
# deduped data

spark.sql("""
with cte as (select *, row_number() over (partition by actorid, year, filmid order by year) as rn
from actor_films)
select actorid, year, film, votes, rating, filmid from cte where rn = 1
""").show(10)

In [ ]:
spark.stop()